In [244]:
import pandas as pd
import scipy as sp

In [170]:
phi_business= pd.read_csv("phi_business.csv")
phi_review=pd.read_csv("phi_review.csv")
phi_category=pd.read_csv("philly_categories_v2.csv")[["business_id","category_kmeans_v1"]]

In [171]:
phi_category=phi_category.rename(columns={"category_kmeans_v1":"category"})
phi_category

,business_id,category
0,MTSW4McQd7CbVtyjqoe9mw,1.0
1,MUTTqe8uqyMdBl186RmNeA,4.0
2,ROeacJQwBeh05Rqg7F6TCg,0.0
3,QdN72BWoyFypdGJhhI5r7g,4.0
4,Mjboz24M9NlBeiOJKLEd_Q,0.0
...,...,...
14603,wPAaAbFbq5wfLfW0gnYU7g,5.0
14604,JDM7kiCEmeBc_5W-z6eZZw,5.0
14605,9U1Igcpe954LoWZRmNc-zg,2.0
14606,LJ4GjQ1HL6kqvIPpNUNNaQ,3.0


In [8]:
phi_review["year"]=phi_review["date"].apply(lambda x: x[0:4])

In [24]:
business_year_open=phi_review.groupby("business_id").min().reset_index()[["business_id","year"]]

In [177]:
business_year_open["year"]= business_year_open["year"].apply(lambda x:int(x))
business_category_year_open=business_year_open.merge(phi_category,on="business_id")
business_category_year_open

,business_id,year,category
0,--OS_I7dnABrXvRCCuWOGQ,2016,3.0
1,-0M0b-XhtFagyLmsBtOe8w,2012,4.0
2,-0PN_KFPtbnLQZEeb23XiA,2009,0.0
3,-0TffRSXXIlBYVbb5AwfTg,2013,4.0
4,-0eUa8TsXFFy0FCxHYmrjg,2015,5.0
...,...,...,...
14603,zyge4T5eSiPHq1-IaJb_Qg,2011,2.0
14604,zyghhZzPgb1bRAIYB-oi1w,2012,3.0
14605,zz-fcqurtm77bZ_rVvo2Lw,2011,1.0
14606,zz3E7kmJI2r2JseE6LAnrw,2007,1.0


In [178]:
business_year_open_post_2020= business_category_year_open [business_category_year_open["year"]>=2020]
business_year_open_pre_2020= business_category_year_open [business_category_year_open["year"]<2020]

In [179]:
business_year_open_post_2020_zip=business_year_open_post_2020.merge(phi_business, on="business_id")[["business_id","year","postal_code","category"]]
business_year_open_post_2020_zip

,business_id,year,postal_code,category
0,-4N4YEobjG89VQsEgqZshg,2020,19147,0.0
1,-RpUSbVtSGO9LZwMzGogww,2021,19125,4.0
2,-SkwJp45TBbqxYzuYHcerA,2020,19147,1.0
3,-TW-0EqgnzUNHzV9VPSLoQ,2021,19114,0.0
4,-mc0GB26tKLQhxKpuR3xUw,2021,19106,1.0
...,...,...,...,...
533,zRnFxKLKdn0ZJ2_DapEQWA,2020,19123,0.0
534,zeG4Jb_ORGiqnGmcww_a4w,2020,19145,4.0
535,zhFONN1m6MzdVcK8QjEljA,2020,19127,5.0
536,zkrKSW5yrb0OPvhwpn-SeA,2020,19106,2.0


In [190]:
post_2020_new_business=business_year_open_post_2020_zip.groupby(["postal_code","category"]).count().reset_index()[["postal_code","category","business_id"]]
post_2020_new_business=post_2020_new_business.rename(columns={"business_id":"post_2020"})
post_2020_new_business=post_2020_new_business.sort_values("postal_code")
post_2020_new_business

,postal_code,category,post_2020
0,19019,5.0,1
1,19092,5.0,1
2,19102,0.0,6
3,19102,1.0,4
4,19102,4.0,2
...,...,...,...
162,19154,2.0,1
163,19154,3.0,1
160,19154,0.0,2
161,19154,1.0,2


In [184]:
business_year_open_pre_2020_zip=business_year_open_pre_2020.merge(phi_business, on="business_id")[["business_id","year","postal_code","category"]]
pre_2020_new_business=business_year_open_pre_2020_zip.groupby(["postal_code","category"]).count().reset_index()[["postal_code","category","business_id"]]
pre_2020_new_business=pre_2020_new_business.rename(columns={"business_id":"pre_2020"})

In [185]:
pre_2020_new_business

,postal_code,category,pre_2020
0,19019,1.0,4
1,19019,2.0,1
2,19019,3.0,8
3,19019,5.0,18
4,19092,1.0,1
...,...,...,...
307,19195,1.0,7
308,19195,2.0,1
309,19195,3.0,5
310,19195,4.0,8


In [208]:
businesses_by_year_category=post_2020_new_business.merge(pre_2020_new_business, on = ["postal_code","category"],how="outer")

In [209]:
businesses_by_year_category=businesses_by_year_category.sort_values(by="postal_code")
businesses_by_year_category=businesses_by_year_category.fillna(0)
#pd.set_option('display.max_rows', 20)
businesses_by_year_category

,postal_code,category,post_2020,pre_2020
0,19019,1.0,0.0,4.0
1,19019,2.0,0.0,1.0
2,19019,3.0,0.0,8.0
3,19019,5.0,1.0,18.0
4,19092,1.0,0.0,1.0
...,...,...,...,...
307,19195,0.0,0.0,10.0
308,19195,1.0,0.0,7.0
309,19195,2.0,0.0,1.0
310,19195,3.0,0.0,5.0


In [204]:
phi_demand_score= pd.read_csv("phi_demand_score.csv")[["business_id","postal_code","total_demand_score","missing_demand_score"]]

In [206]:
phi_demand_score_category=phi_demand_score.merge(phi_category) 
phi_demand_score_category_zip=phi_demand_score_category.groupby(["postal_code","category"]).sum().reset_index()[["postal_code","category","total_demand_score","missing_demand_score"]]
phi_demand_score_category_zip

,postal_code,category,total_demand_score,missing_demand_score
0,19019,1.0,71.0,23
1,19019,2.0,0.0,0
2,19019,3.0,1001.0,426
3,19019,5.0,0.0,244
4,19092,1.0,25.0,3
...,...,...,...,...
308,19195,1.0,1133.0,293
309,19195,2.0,0.0,0
310,19195,3.0,4330.0,595
311,19195,4.0,1695.0,1258


In [229]:
businesses_by_year_category_demand= businesses_by_year_category.merge(phi_demand_score_category_zip,on=["postal_code","category"])
businesses_by_year_category_demand=businesses_by_year_category_demand.sort_values("post_2020",ascending=False).reset_index(drop=True)
businesses_by_year_category_demand

,postal_code,category,post_2020,pre_2020,total_demand_score,missing_demand_score
0,19147,1.0,18.0,220.0,156373.0,23463
1,19147,0.0,13.0,244.0,165349.0,36860
2,19107,1.0,13.0,300.0,255699.0,28114
3,19148,0.0,12.0,110.0,43419.0,9182
4,19107,0.0,12.0,225.0,235429.0,40246
...,...,...,...,...,...,...
308,19131,2.0,0.0,17.0,929.0,382
309,19132,1.0,0.0,15.0,2357.0,908
310,19132,2.0,0.0,3.0,275.0,125
311,19132,3.0,0.0,5.0,600.0,122


In [230]:
new_business_rank= list(businesses_by_year_category_demand.index)
missing_demand_rank=list(businesses_by_year_category_demand.sort_values(by="missing_demand_score",ascending=False).index)
total_demand_rank=list(businesses_by_year_category_demand.sort_values(by="total_demand_score",ascending=False).index)

In [245]:
sp.stats.kendalltau(new_business_rank, missing_demand_rank)

SignificanceResult(statistic=0.44966003112967967, pvalue=1.765518813051642e-32)